In [1]:
import os

In [2]:
%pwd

'c:\\Users\\nguye\\Documents\\AI\\Youtube_tutorials\\end-to-end-ml-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\nguye\\Documents\\AI\\Youtube_tutorials\\end-to-end-ml-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from ml_in_action.constants import *
from ml_in_action.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
import os
from ml_in_action import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [9]:
class DataTransformation:
    def __init__(self, config):
        self.config = config

    def train_test_spliting(self):
        # Load dataset
        data = pd.read_csv(self.config.data_path)

        # Drop the last column (assuming it's named 'Id')
        if 'Id' in data.columns:
            data = data.drop(columns=['Id'])
        
        # Rename columns to better recall them.
        data.rename(columns = {"fixed acidity": "fixed_acidity",
                            "volatile acidity": "volatile_acidity",
                            "citric acid": "citric_acid",
                            "residual sugar": "residual_sugar",
                            "chlorides": "chlorides",
                            "free sulfur dioxide": "free_sulfur_dioxide",
                            "total sulfur dioxide": "total_sulfur_dioxide"},
                    inplace = True)

        # Replace with the label Bad, Middle, Good
        data = data.replace({'quality': {
            8: 'Good',
            7: 'Good',
            6: 'Middle',
            5: 'Middle',
            4: 'Bad',
            3: 'Bad',
        }})
        
        # Data Regularization
        X_temp = data.drop(columns='quality')
        y = data.quality
        
        scaler = MinMaxScaler(feature_range=(0, 1)).fit_transform(X_temp)
        X = pd.DataFrame(scaler, columns=X_temp.columns)

        # Train-test split (80% train, 20% test)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Convert back to DataFrame
        train = pd.DataFrame(X_train, columns=X.columns)
        train['quality'] = y_train.values  # Add target back

        test = pd.DataFrame(X_test, columns=X.columns)
        test['quality'] = y_test.values  # Add target back

        # Save processed data
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split data into training and test sets")
        logger.info(f"Train shape: {train.shape}, Test shape: {test.shape}")

        print(f"Train shape: {train.shape}, Test shape: {test.shape}")

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-03-16 16:28:40,540: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-16 16:28:40,542: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 16:28:40,544: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-16 16:28:40,545: INFO: common: created directory at: artifacts]
[2025-03-16 16:28:40,547: INFO: common: created directory at: artifacts/data_transformation]
[2025-03-16 16:28:40,568: INFO: 3741347456: Split data into training and test sets]
[2025-03-16 16:28:40,569: INFO: 3741347456: Train shape: (914, 12), Test shape: (229, 12)]
Train shape: (914, 12), Test shape: (229, 12)
